# Custom Model Function Calling Evaluation Demo

**🎯 Goal**:
- Run a function calling evaluation in Okareo.
- Provide a simple introduction to Okareo evaluations.

**📋 Steps**:
1. Upload a function calling scenario.
2. Define a custom model to generate the function calls
3. Run the evaluation using the scenario (from #1) + model (from #2) along with a check to measure function call accuracy.

In [ ]:
# install okareo
%pip install okareo

In [1]:
import os

# get Okareo client
from okareo import Okareo

OKAREO_API_KEY = os.environ.get("OKAREO_API_KEY")
if not OKAREO_API_KEY:
    raise ValueError("OKAREO_API_KEY environment variable is not set")
okareo = Okareo(OKAREO_API_KEY)

Upload a simple scenario. Each row of the `seed_data` should contain:

- `input_`: query to the agent.
- `result`: the expected function call that answers the query.

Note: In the `result` function call, the `__required` key is an Okareo-specific key needed to use the `are_required_params_present` check. 

In [53]:
import random
import string

from okareo_api_client.models.scenario_set_create import ScenarioSetCreate
from okareo_api_client.models.seed_data import SeedData

def random_string(length: int) -> str:
    return "".join(random.choices(string.ascii_letters, k=length))

seed_data = [
    SeedData(
        input_="can you delete my account? my name is Bob",
        result={"function": {"name": "delete_account", "arguments": { "username": "Bob" }, "__required": ["username"]}},
    ),
    SeedData(
        input_="can you delete my account? my name is john",
        result={"function": {"name": "delete_account", "arguments": { "username": "John" }, "__required": ["username"]}},
    ),
    SeedData(
        input_="how do I make an account? my name is Alice",
        result={"function": {"name": "create_account", "arguments": { "username": "Alice"}, "__required": ["username"]}},
    ),
    SeedData(
        input_="how do I create an account?",
        result={"function": {"name": "create_account", "arguments": { "username": ".+" }, "__required": ["username"]}},
    ),
    SeedData(
        input_="my name is steve. how do I create a project?",
        result={"function": {"name": "create_account", "arguments": { "username": "Steve" }, "__required": ["username"]}},
    ),
]

tool_scenario = okareo.create_scenario_set(
    ScenarioSetCreate(
        name=f"Function Call Demo Scenario - {random_string(5)}",
        seed_data=seed_data,
    ) 
)

print(tool_scenario.app_link)

http://localhost:3000/project/cb832f0d-0e3d-4bed-9d9c-023441ec9dd5/scenario/6ae97dad-dfd0-4a75-b4e2-db1c6a6ef634


Define the [CustomModel](https://docs.okareo.ai/docs/sdk/okareo_python#custommodel--modelinvocation) using conditionals, and register the model with Okareo.

In reality, you would use the `CustomModel` class to invoke and parse your LLM's outputs. Feel free to play around!

In [54]:
import re
from okareo.model_under_test import CustomModel, ModelInvocation

class FunctionCallModel(CustomModel):
    def __init__(self, name):
        super().__init__(name)
        self.usernames = ["Bob", "Alice", "John"]
        self.pattern = r'my name is (\S+)'

    def invoke(self, input_value):
        out = {"tool_calls": []}
        function_call = {"name": "unknown"}

        # parse out the function name
        if "delete" in input_value:
            function_call["name"] = "delete_account"
        if "create" in input_value:
            function_call["name"] = "create_account"

        # parse out the function argument
        match = re.search(self.pattern, input_value)
        if match:
            username = match.group(1)
            function_call["arguments"] = {"username": username}

        tool_call = {
            "function": function_call
        }

        # package the tool call and return
        out["tool_calls"].append(tool_call)
        return ModelInvocation(
            model_input=input_value,
            tool_calls=out["tool_calls"]
        )

# Register the model to use in the test run
mut_name="Function Call Demo Model"
model_under_test = okareo.register_model(
    name=mut_name,
    model=[FunctionCallModel(name=FunctionCallModel.__name__)],
    update=True
)

Run a [Generation evaluation](https://docs.okareo.ai/docs/guides/generation_overview) on the custom model. 

We use a predefined [check in Okareo](https://docs.okareo.ai/docs/getting-started/concepts/checks) called function_call_validator, which uses regular expressions to match the expected result with the tool call.

In [60]:
# evaluation that uses the scenario, check, and model
from okareo_api_client.models.test_run_type import TestRunType

eval_name = f"Function Call Demo Evaluation"
evaluation = model_under_test.run_test(
    name=eval_name,
    scenario=tool_scenario.scenario_id,
    test_run_type=TestRunType.NL_GENERATION,
    checks=["function_call_validator",
            "is_function_correct",
            "are_all_params_expected",
            "are_required_params_present",
            "do_param_values_match"]
)
print(f"See results in Okareo: {evaluation.app_link}")

See results in Okareo: http://localhost:3000/project/cb832f0d-0e3d-4bed-9d9c-023441ec9dd5/eval/0d1f00e2-e12c-4b07-9573-38be0f5eddf0
